<a href="https://colab.research.google.com/github/okechukwuchude/Automating-Medical-Coding/blob/main/Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import os
import json
#import stanza
import csv
import pandas as pd

# Build an English pipeline
#stanza.download('en', package='mimic', processors={'ner': 'i2b2'}) # download English model
#nlp = stanza.Pipeline('en', package='mimic', processors={'ner': 'i2b2'}) # initialize English neural pipeline

In [69]:
token_df = pd.read_csv('/home/chudeo/project/full_tokens.csv')

In [70]:
token_df.head()

,sentence_id,words,labels
0,0,Baseline,O
1,0,artifact,O
2,0,.,O
3,1,Probable,O
4,1,sinus,B


In [71]:
# # Convert all values in the 'word' column to strings
# token_df['word'] = token_df['words'].astype(str)


# Drop rows containing non-string values in the 'words' column
token_df = token_df.dropna(subset=['words'])  # Drop NaN values
token_df = token_df[token_df['words'].apply(lambda x: isinstance(x, str))]  # Filter out non-string values


In [72]:
x = token_df[['sentence_id', 'words']]
y = token_df['labels']

SPLITTING DATA

In [73]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

In [74]:
#build the training and test data
train_data = pd.DataFrame({'sentence_id': x_train['sentence_id'], 'words': x_train['words'], 'labels': y_train})
test_data = pd.DataFrame({'sentence_id': x_test['sentence_id'], 'words': x_test['words'], 'labels': y_test})

In [75]:
train_data

,sentence_id,words,labels
1222807,138,),O
141314,7,:,O
82478,180,First,O
746401,14,hemidiaphragm,O
136123,15,1.5,O
...,...,...,...
1157382,74,ASDIR,O
172635,96,",",O
1497529,29,BRONCHOALVEOLAR,O
1404898,126,Part,O


MODEL TRAINING

BERT

In [76]:
pip install simpletransformers

Note: you may need to restart the kernel to use updated packages.


In [77]:
from simpletransformers.ner import NERModel, NERArgs

In [78]:
label = token_df['labels'].unique().tolist()
label

['O', 'B', 'I']

In [79]:
args = NERArgs()
args.num_train_epochs = 10
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [80]:
model = NERModel('bert', 'bert-base-cased', labels=label, args=args, use_cuda = True)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
# Train the model
model.train_model(train_data, eval_data=test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

/home/chudeo/using_gpu/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 2 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

...

Running Epoch 7 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

Running Epoch 10 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

(170, 0.06701843880548361)

In [82]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/16 [00:00<?, ?it/s]

In [83]:
result

{'eval_loss': 0.056901418953202665,
 'precision': 0.3305785123966942,
 'recall': 0.15810276679841898,
 'f1_score': 0.21390374331550802}

In [84]:
prediction, model_output = model.predict(['There is also intraventricular extension into the ipsilateral and contralateral lateral ventricles.'])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [85]:
prediction

[[{'There': 'O'},
  {'is': 'O'},
  {'also': 'O'},
  {'intraventricular': 'I'},
  {'extension': 'I'},
  {'into': 'I'},
  {'the': 'I'},
  {'ipsilateral': 'I'},
  {'and': 'I'},
  {'contralateral': 'I'},
  {'lateral': 'I'},
  {'ventricles.': 'I'}]]

BioBERT

In [86]:
bio_label = token_df['labels'].unique().tolist()
bio_label

['O', 'B', 'I']

In [87]:
# Initialize NER model arguments
args = NERArgs()
args.num_train_epochs = 10  # Number of training epochs
args.learning_rate = 2e-5   # Learning rate
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [88]:
# Create the NER model with BioBERT
model = NERModel('bert', 'dmis-lab/biobert-base-cased-v1.2', args=args, labels=bio_label, use_cuda=True)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [89]:
# Train the model
model.train_model(train_data)

# Evaluate the model on test data
bio_result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

Running Epoch 3 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

...

Running Epoch 8 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

Running Epoch 10 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/16 [00:00<?, ?it/s]

In [90]:
bio_result

{'eval_loss': 0.03991517031681724,
 'precision': 0.4473684210526316,
 'recall': 0.06719367588932806,
 'f1_score': 0.11683848797250859}

In [91]:

predictions, raw_outputs = model.predict(["T[**Name2 (NI) **] was extubated on [**10-24**] without"])
print(predictions)


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'T[**Name2': 'O'}, {'(NI)': 'O'}, {'**]': 'O'}, {'was': 'O'}, {'extubated': 'O'}, {'on': 'O'}, {'[**10-24**]': 'O'}, {'without': 'O'}]]


Hyperparameter Tuning

In [92]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Build the training, validation, and test data
train_data = pd.DataFrame({'sentence_id': x_train['sentence_id'], 'words': x_train['words'], 'labels': y_train})
val_data = pd.DataFrame({'sentence_id': x_val['sentence_id'], 'words': x_val['words'], 'labels': y_val})
test_data = pd.DataFrame({'sentence_id': x_test['sentence_id'], 'words': x_test['words'], 'labels': y_test})


In [94]:
from sklearn.metrics import accuracy_score
from simpletransformers.ner import NERModel, NERArgs
import logging


# Define hyperparameters to search over
param_grid = {
    'learning_rate': [1e-5, 2e-5, 3e-5],
    'num_train_epochs': [1, 2, 3],
    'train_batch_size': [16, 32, 64]
}

# Initialize your model
model = NERModel('bert', 'dmis-lab/biobert-base-cased-v1.2', args=args, labels=bio_label, use_cuda=True)

best_accuracy = 0
best_params = {}

# Split test_data into batches to avoid memory issues
test_batches = [test_data[i:i+32] for i in range(0, len(test_data), 32)]

# Define early stopping criteria
early_stop_patience = 3
early_stop_counter = 0
best_val_accuracy = 0


# Configure logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler('training.log')
stream_handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s')
file_handler.setFormatter(formatter)
stream_handler.setFormatter(formatter)
logger.addHandler(file_handler)
logger.addHandler(stream_handler)


# Perform manual hyperparameter tuning
for lr in param_grid['learning_rate']:
    for epochs in param_grid['num_train_epochs']:
        for batch_size in param_grid['train_batch_size']:
            logger.info(f"Training with lr={lr}, epochs={epochs}, batch_size={batch_size}")

            # Initialize the model with current hyperparameters
            args = NERArgs()
            args.learning_rate = lr
            args.num_train_epochs = epochs
            args.train_batch_size = batch_size
            model = NERModel('bert', 'dmis-lab/biobert-base-cased-v1.2', args=args, labels=bio_label, use_cuda=True)


            # Train the model
            for epoch in range(epochs):
                model.train_model(train_data)

                # Evaluate on validation set
                _, _, val_preds = model.eval_model(val_data)
                val_accuracy = accuracy_score(val_data['labels'], val_preds)

                # Check for early stopping
                if val_accuracy > best_val_accuracy:
                    best_val_accuracy = val_accuracy
                    early_stop_counter = 0
                else:
                    early_stop_counter += 1
                    if early_stop_counter >= early_stop_patience:
                        logger.info(f"Early stopping after {epoch + 1} epochs.")
                        break

            # Evaluate on test set
            _, _, test_preds = model.eval_model(test_data)
            accuracy = accuracy_score(test_data['labels'], test_preds)
            logger.info(f"Accuracy: {accuracy:.4f}")

            # Update best parameters and accuracy
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {'learning_rate': lr, 'num_train_epochs': epochs, 'train_batch_size': batch_size}

# Log best hyperparameters and accuracy
logger.info("Best Parameters: %s", str(best_params))
logger.info("Best Accuracy: %.4f", best_accuracy)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2024-03-27 22:41:21,557 [INFO] Training with lr=1e-05, epochs=1, batch_size=16
2024-03-27 22:41:21,557 [INFO] Training with lr=1e-05, epochs=1, batch_size=16
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

/home/chudeo/using_gpu/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ValueError: Found input variables with inconsistent numbers of samples: [211792, 498]

Regularization

In [95]:

# Define regularization parameters
args = NERArgs()
args.num_train_epochs = 5
args.learning_rate = 2e-5
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32
args.weight_decay = 0.01  # Example regularization parameter

# Initialize the model with regularization
model = NERModel('bert', 'dmis-lab/biobert-base-cased-v1.2', args=args, labels=bio_label, use_cuda=False)

# Train the model
model.train_model(train_data)

# Evaluate the model
result, model_outputs, preds_list = model.eval_model(test_data)
print(result)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/16 [00:00<?, ?it/s]

...

Running Epoch 4 of 5:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/16 [00:00<?, ?it/s]

{'eval_loss': 0.046454982104478404, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}


In [96]:
result

{'eval_loss': 0.046454982104478404,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

Fine-tuning BERT for named-entity recognition

In [97]:
token_df

,sentence_id,words,labels
0,0,Baseline,O
1,0,artifact,O
2,0,.,O
3,1,Probable,O
4,1,sinus,B
...,...,...,...
1513257,134,(,O
1513258,134,2,O
1513259,134,),O
1513260,134,7284,O


In [98]:

label2id = {k: v for v, k in enumerate(token_df.labels.unique())}
id2label = {v: k for v, k in enumerate(token_df.labels.unique())}
label2id

{'O': 0, 'B': 1, 'I': 2}

#### **Preparing the dataset and dataloader**

In [99]:
from transformers import BertTokenizer, BertForMaskedLM

In [100]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [101]:
import torch
from torch.utils.data import Dataset

class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        words = self.data.iloc[index]['words'].split(",")  # Assuming tokenized words are separated by comma
        labels = self.data.iloc[index]['labels'].split(",")

        # Convert tokenized words to token IDs
        input_ids = [self.tokenizer.convert_tokens_to_ids(word.split()) for word in words]

        # Pad or truncate tokenized words and labels to fit MAX_LEN
        if len(input_ids) > self.max_len:
            input_ids = input_ids[:self.max_len]
            labels = labels[:self.max_len]
        elif len(input_ids) < self.max_len:
            padding_length = self.max_len - len(input_ids)
            input_ids += [[0]] * padding_length  # Padding token ID assumed to be 0
            labels += ['O'] * padding_length

        # Flatten the list of token IDs
        input_ids = [token_id for sublist in input_ids for token_id in sublist]

        # Convert labels to label ids
        label_ids = [label2id[label] for label in labels]

        # Create attention mask
        attention_mask = [1 if token_id != 0 else 0 for token_id in input_ids]  # Assuming padding token ID is 0

        return {
            'ids': torch.tensor(input_ids, dtype=torch.long),
            'mask': torch.tensor(attention_mask, dtype=torch.long),
            'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len


In [102]:
# from torch.nn.utils.rnn import pad_sequence

# def collate_fn(batch):
#     # Get input_ids, masks, and targets from the batch
#     ids = [sample['ids'] for sample in batch]
#     masks = [sample['mask'] for sample in batch]
#     targets = [sample['targets'] for sample in batch]

#     # Pad sequences to the length of the longest sequence in the batch
#     ids = pad_sequence(ids, batch_first=True, padding_value=tokenizer.pad_token_id)
#     masks = pad_sequence(masks, batch_first=True, padding_value=0)  # Padding token ID is assumed to be 0
#     targets = pad_sequence(targets, batch_first=True, padding_value=-100)  # Replace -100 with appropriate padding value for labels

#     # Ensure alignment between input_ids, masks, and targets
#     masks = masks.bool()  # Convert to boolean mask
#     targets = targets * masks.long()  # Apply mask to targets

#     return {
#         'ids': ids,
#         'mask': masks,
#         'targets': targets
#     }


In [103]:
def collate_fn(batch):
    ids = [sample['ids'] for sample in batch]
    masks = [sample['mask'] for sample in batch]
    targets = [sample['targets'] for sample in batch]

    # Pad sequences dynamically to the maximum length within the batch
    max_len = max(len(seq) for seq in ids)
    ids = [torch.nn.functional.pad(seq, (0, max_len - len(seq)), value=0) for seq in ids]
    masks = [torch.nn.functional.pad(seq, (0, max_len - len(seq)), value=0) for seq in masks]
    targets = [torch.nn.functional.pad(seq, (0, max_len - len(seq)), value=-100) for seq in targets]

    ids = torch.stack(ids, dim=0)
    masks = torch.stack(masks, dim=0)
    targets = torch.stack(targets, dim=0)

    return {
        'ids': ids,
        'mask': masks,
        'targets': targets
    }


In [104]:
# Define train and test datasets using train_test_split
train_df, test_df = train_test_split(token_df, test_size=0.2, random_state=42)

print("FULL Dataset: {}".format(token_df.shape))
print("TRAIN Dataset: {}".format(train_df.shape))
print("TEST Dataset: {}".format(test_df.shape))

# Create dataset instances
training_set = dataset(train_df, tokenizer, MAX_LEN)
testing_set = dataset(test_df, tokenizer, MAX_LEN)

FULL Dataset: (1512795, 3)
TRAIN Dataset: (1210236, 3)
TEST Dataset: (302559, 3)


In [105]:
training_set[0]

{'ids': tensor([100,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0]),
 'mask': tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0

In [106]:
training_set[0]["ids"]

tensor([100,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0])

In [107]:
# print the first 30 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:30]), training_set[0]["targets"][:30]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

[UNK]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O


Now, let's define the corresponding PyTorch dataloaders:

In [108]:
from torch.utils.data import Dataset, DataLoader

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

# training_loader = DataLoader(training_set, **train_params)
# testing_loader = DataLoader(testing_set, **test_params)

training_loader = DataLoader(training_set, collate_fn=collate_fn, **train_params)
testing_loader = DataLoader(testing_set, collate_fn=collate_fn, **test_params)

#### **Defining the model**

In [109]:
from transformers import BertTokenizer, BertConfig, BertForTokenClassification


from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [110]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', 
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

#### **Training the model**


In [111]:
from sklearn.metrics import accuracy_score


In [112]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(1.5896, device='cuda:0', grad_fn=<NllLossBackward0>)

In [113]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 3])

In [114]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [115]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [116]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 1.0059914588928223
Training loss per 100 training steps: 0.1206019668895198
Training loss per 100 training steps: 0.06339988990835105
Training loss per 100 training steps: 0.04348713418923665
Training loss per 100 training steps: 0.03345326216603418
Training loss per 100 training steps: 0.027170831256243075
Training loss per 100 training steps: 0.023015102398255993
Training loss per 100 training steps: 0.019953243587629076
Training loss per 100 training steps: 0.017570019693601616
Training loss per 100 training steps: 0.015828961418689922
Training loss per 100 training steps: 0.014477360259951349
Training loss per 100 training steps: 0.0132739151566876
Training loss per 100 training steps: 0.012310035926197092
Training loss per 100 training steps: 0.011576611617931987
Training loss per 100 training steps: 0.010869534927597143
Training loss per 100 training steps: 0.010281310765121798
Training loss per 100 training steps: 0.0097118

#### **Evaluating the model**

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    
    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
labels, predictions = valid(model, testing_loader)

In [ ]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))